# 📘 Agentic Architectures 4: Planning

In this notebook, we explore the **Planning** architecture. This pattern introduces a crucial layer of foresight into an agent's reasoning process. Instead of reacting to information step-by-step as in the ReAct model, a planning agent first decomposes a complex task into a sequence of smaller, manageable sub-goals. It creates a full 'battle plan' *before* taking any action.

This proactive approach brings structure, predictability, and efficiency to multi-step tasks. To highlight its benefits, we will directly compare the performance of a **reactive agent (ReAct)** against our new **planning agent**. We will present both with a task that requires gathering multiple pieces of information before performing a final calculation, demonstrating how a pre-computed plan can lead to a more robust and direct solution.

### Definition
The **Planning** architecture involves an agent that explicitly breaks down a complex goal into a detailed sequence of sub-tasks *before* beginning execution. The output of this initial planning phase is a concrete, step-by-step plan that the agent then follows methodically to reach the solution.

### High-level Workflow

1.  **Receive Goal:** The agent is given a complex task.
2.  **Plan:** A dedicated 'Planner' component analyzes the goal and generates an ordered list of sub-tasks required to achieve it. For example: `["Find fact A", "Find fact B", "Calculate C using A and B"]`.
3.  **Execute:** An 'Executor' component takes the plan and carries out each sub-task in sequence, using tools as needed.
4.  **Synthesize:** Once all steps in the plan are complete, a final component synthesizes the results from the executed steps into a coherent final answer.

### When to Use / Applications
*   **Multi-Step Workflows:** Ideal for tasks where the sequence of operations is known and critical, such as generating a report that requires fetching data, processing it, and then summarizing it.
*   **Project Management Assistants:** Decomposing a large goal like "launch a new feature" into sub-tasks for different teams.
*   **Educational Tutoring:** Creating a lesson plan to teach a student a specific concept, from foundational principles to advanced application.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Structured & Traceable:** The entire workflow is laid out in advance, making the agent's process transparent and easy to debug.
    *   **Efficient:** Can be more efficient than ReAct for predictable tasks, as it avoids unnecessary reasoning loops between steps.
*   **Weaknesses:**
    *   **Brittle to Change:** A pre-made plan can fail if the environment changes unexpectedly during execution. It's less adaptive than a ReAct agent, which can change its mind after every step.

## Phase 0: Foundation & Setup

We'll begin with our standard setup process: installing libraries and configuring API keys for Together, LangSmith, and our Tavily web search tool.

### Step 0.1: Installing Core Libraries

**What we are going to do:**
We will install our standard suite of libraries, including the updated `langchain-tavily` package to address the deprecation warning.

In [1]:
# !pip install -q -U langchain-together langchain langgraph rich python-dotenv langchain-tavily

### Step 0.2: Importing Libraries and Setting Up Keys

**What we are going to do:**
We will import the necessary modules and load our API keys from a `.env` file.

In [2]:
import os
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_together import ChatTogether
from langchain_core.messages import BaseMessage, ToolMessage
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langchain_tavily import TavilySearch

# LangGraph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "04_TIDIT_Workshop"

# Check that the keys are set
for key in ["TOGETHER_API_KEY", "LANGSMITH_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: The Baseline - A Reactive Agent (ReAct)

To appreciate the value of planning, we first need a baseline. We will use the ReAct agent we built in the previous notebook. This agent is intelligent but myopic—it figures out its path one step at a time.

### Step 1.1: Re-building the ReAct Agent

**What we are going to do:**
We will quickly reconstruct the ReAct agent. Its core feature is a loop where the agent's output is routed back to itself after every tool call, allowing it to reassess and decide its next move based on the latest information.

In [ ]:
console = Console()

# Define the state for our graphs
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# 1. Define the base tool from the tavily package
tavily_search_tool = TavilySearch(max_results=2, tavily_api_key=os.environ["TAVILY_API_KEY"])

# 2. THE FIX: Simplify the custom tool. 
#    The .invoke() method already returns a clean string, so we just pass it through.
@tool
def web_search(query: str) -> str:
    """Performs a web search using Tavily and returns the results as a string."""
    console.print(f"--- TOOL: Searching for '{query}'...")
    results = tavily_search_tool.invoke(query)
    return results

# 3. Define the LLM and bind it to our custom tool
llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature=0
)
llm_with_tools = llm.bind_tools([web_search])

# 4. Agent node with a system prompt to force one tool call at a time
def react_agent_node(state: AgentState):
    console.print("--- REACTIVE AGENT: Thinking... ---")
    
    messages_with_system_prompt = [
        SystemMessage(content="You are a helpful research assistant. You must call one and only one tool at a time. Do not call multiple tools in a single turn. After receiving the result from a tool, you will decide on the next step.")
    ] + state["messages"]

    response = llm_with_tools.invoke(messages_with_system_prompt)
    
    return {"messages": [response]}

# 5. Use our corrected custom tool in the ToolNode
tool_node = ToolNode([web_search])

# The ReAct graph with its characteristic loop
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", tool_node)
react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", tools_condition)
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("Reactive (ReAct) agent compiled successfully.")

Reactive (ReAct) agent compiled successfully.


### Step 1.2: Testing the Reactive Agent on a Plan-Centric Problem

**What we are going to do:**
We will give the ReAct agent a task that requires two distinct data-gathering steps followed by a final calculation. This will test its ability to manage a multi-step workflow without an upfront plan.

In [4]:
plan_centric_query = """
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
"""

console.print(f"[bold yellow]Testing REACTIVE agent on a plan-centric query:[/bold yellow] '{plan_centric_query}'\n")

final_react_output = None
for chunk in react_agent_app.stream({"messages": [("user", plan_centric_query)]}, stream_mode="values"):
    final_react_output = chunk
    console.print(f"--- [bold purple]Current State Update[/bold purple] ---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold red]Final Output from Reactive Agent[/bold red] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

Testing REACTIVE agent on a plan-centric query: '
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
'

--- Current State Update ---

================================ Human Message =================================


Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.



--- REACTIVE AGENT: Thinking... ---

--- Current State Update ---

...

--- REACTIVE AGENT: Thinking... ---

--- Current State Update ---

================================== Ai Message ==================================

The population of Paris is approximately 11.2 million, Berlin is around 3.6 million, and Rome is around 4.3 million (though the source provided a range, we will use the higher estimate for this calculation). The combined total of these cities is 11.2 + 3.6 + 4.3 = 19.1 million. 

The population of the United States is approximately 331 million. 

Comparing the two, the population of the United States (331 million) is larger than the combined total of the capital cities of France, Germany, and Italy (19.1 million).


--- Final Output from Reactive Agent ---

The population of Paris is approximately 11.2 million, Berlin is around 3.6 million, and Rome is around 4.3 million
(though the source provided a range, we will use the higher estimate for this calculation). The combined total of  
these cities is 11.2 + 3.6 + 4.3 = 19.1 million.                                                                   

The population of the United States is approximately 331 million.                                                  

Comparing the two, the population of the United States (331 million) is larger than the combined total of the      
capital cities of France, Germany, and Italy (19.1 million).

**Discussion of the Output:**
The ReAct agent successfully completed the task. By observing the streamed output, we can trace its step-by-step reasoning process:
1. It first decides to search for the population of Paris.
2. After receiving that result, it incorporates it into its memory and then decides its next step is to search for the population of Berlin.
3. Finally, with both pieces of information gathered, it performs the calculation and provides the final answer.

While it works, this iterative discovery process is not always the most efficient. For a predictable task like this, the agent is making extra LLM calls to reason between each step. This sets the stage for demonstrating the value of a planning agent.

## Phase 2: The Advanced Approach - A Planning Agent

Now, let's build an agent that thinks before it acts. This agent will have a dedicated **Planner** to create a complete task list, an **Executor** to carry out the plan, and a **Synthesizer** to assemble the final result.

### Step 2.1: Defining the Planner, Executor, and Synthesizer Nodes

**What we are going to do:**
We will create the core components for our new agent:
1.  **`Planner`:** An LLM-based node that takes the user request and outputs a structured plan.
2.  **`Executor`:** A node that takes the plan, executes the *next* step using a tool, and records the result.
3.  **`Synthesizer`:** A final LLM-based node that takes all the collected results and generates the final answer.

In [5]:
class PlanStep(BaseModel):
    """A single step in the plan."""
    tool: str = Field(description="The name of the tool to call, e.g. 'web_search'.")
    query: str = Field(description="The search query to pass to the tool.")

class Plan(BaseModel):
    """A plan of tool calls to execute to answer the user's query."""
    steps: List[PlanStep] = Field(description="An ordered list of tool calls, each with a tool name and query.")

class PlanningState(TypedDict):
    user_request: str
    plan: Optional[list]
    initial_plan: Optional[list]
    intermediate_steps: List[ToolMessage]
    final_answer: Optional[str]

def planner_node(state: PlanningState):
    """Generates a plan of action to answer the user's request."""
    console.print("--- PLANNER: Decomposing task... ---")
    planner_llm = llm.with_structured_output(Plan)
    
    prompt = f"""You are an expert planner. Your job is to create a step-by-step plan to answer the user's request.
        Each step in the plan must be a single call to the `web_search` tool.

        **Instructions:**
        1. Analyze the user's request.
        2. Break it down into a sequence of simple, logical search queries.
        3. Return a structured list of steps, each with the tool name and query string.

        **User's Request:**
        {state['user_request']}
    """

    plan_result = planner_llm.invoke(prompt)
    console.print(f"--- PLANNER: Generated Plan: {[s.model_dump() for s in plan_result.steps]} ---")
    return {"plan": plan_result.steps, "initial_plan": plan_result.steps}

def executor_node(state: PlanningState):
    """Executes the next step in the plan."""
    console.print("--- EXECUTOR: Running next step... ---")
    plan = state["plan"]
    next_step = plan[0]
    
    console.print(f"--- EXECUTOR: Calling tool '{next_step.tool}' with query '{next_step.query}' ---")
    
    result = tavily_search_tool.invoke(next_step.query)
    
    tool_message = ToolMessage(
        content=str(result),
        name=next_step.tool,
        tool_call_id=f"manual-{hash(next_step.query)}"
    )
    
    return {
        "plan": plan[1:],
        "intermediate_steps": state["intermediate_steps"] + [tool_message]
    }

def synthesizer_node(state: PlanningState):
    """Synthesizes the final answer from the intermediate steps."""
    console.print("--- SYNTHESIZER: Generating final answer... ---")
    
    context = "\n".join([f"Tool {msg.name} returned: {msg.content}" for msg in state["intermediate_steps"]])
    
    prompt = f"""You are an expert synthesizer. Based on the user's request and the collected data, provide a comprehensive final answer.
    
    Request: {state['user_request']}
    Collected Data:
    {context}
    """
    final_answer = llm.invoke(prompt).content
    return {"final_answer": final_answer}

print("Planner, Executor, and Synthesizer nodes defined.")

Planner, Executor, and Synthesizer nodes defined.


### Step 2.2: Building the Planning Agent Graph

**What we are going to do:**
Now we will assemble the new nodes into a graph. The flow will be: `Planner` -> `Executor` (looped) -> `Synthesizer`.

In [6]:
def planning_router(state: PlanningState):
    if not state["plan"]:
        console.print("--- ROUTER: Plan complete. Moving to synthesizer. ---")
        return "synthesizer"
    else:
        console.print("--- ROUTER: Plan has more steps. Continuing execution. ---")
        return "executor"

# Node names must not match state keys (e.g. "plan"), so we use planner/executor/synthesizer
planning_graph_builder = StateGraph(PlanningState)
planning_graph_builder.add_node("planner", planner_node)
planning_graph_builder.add_node("executor", executor_node)
planning_graph_builder.add_node("synthesizer", synthesizer_node)

planning_graph_builder.set_entry_point("planner")
planning_graph_builder.add_conditional_edges("planner", planning_router, {"executor": "executor", "synthesizer": "synthesizer"})
planning_graph_builder.add_conditional_edges("executor", planning_router, {"executor": "executor", "synthesizer": "synthesizer"})
planning_graph_builder.add_edge("synthesizer", END)

planning_agent_app = planning_graph_builder.compile()
print("Planning agent compiled successfully.")

Planning agent compiled successfully.


## Phase 3: Head-to-Head Comparison

Let's run our new planning agent on the same task and compare its execution flow and final output to the reactive agent.

In [ ]:
console.print(f"[bold green]Testing PLANNING agent on the same plan-centric query:[/bold green] '{plan_centric_query}'\n")

# Remember to initialize the state correctly, especially the list for intermediate steps
initial_planning_input = {"user_request": plan_centric_query, "intermediate_steps": []}

final_planning_output = planning_agent_app.invoke(initial_planning_input)

console.print("\n--- [bold green]Final Output from Planning Agent[/bold green] ---")
console.print(Markdown(final_planning_output['final_answer']))

Testing PLANNING agent on the same plan-centric query: '
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
'

--- PLANNER: Decomposing task... ---

--- PLANNER: Generated Plan: [{'tool': 'web_search', 'query': 'population of Paris'}, {'tool': 'web_search', 
'query': 'population of Berlin'}, {'tool': 'web_search', 'query': 'population of Rome'}, {'tool': 'web_search', 
'query': 'population of United States'}] ---

--- ROUTER: Plan has more steps. Continuing execution. ---

--- EXECUTOR: Running next step... ---

...

--- ROUTER: Plan has more steps. Continuing execution. ---

--- EXECUTOR: Running next step... ---

--- EXECUTOR: Calling tool 'web_search' with query 'population of United States' ---

--- ROUTER: Plan complete. Moving to synthesizer. ---

--- SYNTHESIZER: Generating final answer... ---

**Discussion of the Output:**
The difference in process is immediately clear. The very first step was the `Planner` creating a complete, explicit plan — a list of structured steps, each specifying the tool to call and the query to pass to it (e.g. `{"tool": "web_search", "query": "population of Paris"}`).

The agent then executed this plan methodically without needing to stop and think between steps. This process is:
- **More Transparent:** We can see the agent's entire strategy before it even starts.
- **More Robust:** It's less likely to get sidetracked because it's following a clear set of instructions.
- **Potentially More Efficient:** It avoids extra LLM calls for reasoning between steps.

This demonstrates the power of planning for tasks where the required steps can be determined in advance.

## Phase 4: Quantitative Evaluation

To formalize our comparison, we will use an LLM-as-a-Judge to score both agents, focusing on the quality and efficiency of their problem-solving process.

In [ ]:
class ProcessEvaluation(BaseModel):
    """Schema for evaluating an agent's problem-solving process."""
    task_completion_score: int = Field(description="Score 1-10 on whether the agent successfully completed the task.")
    process_efficiency_score: int = Field(description="Score 1-10 on the efficiency and directness of the agent's process. A higher score means a more logical and less roundabout path.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(ProcessEvaluation)

def evaluate_agent_process(query: str, final_state: dict):
    # For the ReAct agent, the trace is in 'messages'. For Planning, show the initial_plan (laid out before execution) and then results.
    if 'messages' in final_state:
        trace = "\n".join([f"{m.type}: {str(m.content)}" for m in final_state['messages']])
    else:
        initial_plan = final_state.get('initial_plan', []) or []
        steps = final_state.get('intermediate_steps', [])
        plan_str = "\n".join([f"  {i+1}. {s.tool}(\"{s.query}\")" for i, s in enumerate(initial_plan)]) if initial_plan else "(none)"
        steps_str = "\n".join([f"  Step {i+1}: {getattr(s, 'name', 'tool')} -> (result length {len(getattr(s, 'content', '') or '')} chars)" for i, s in enumerate(steps)])
        trace = f"Agent type: Planning. This agent FIRST laid out the full plan below, THEN executed each step in order with no further reasoning.\n\nInitial plan (created before any execution):\n{plan_str}\n\nExecution (each step run in sequence):\n{steps_str}"
        
    prompt = f"""You are an expert judge of AI agents. Evaluate the agent's process for solving the task on a scale of 1-10.
    
    **Scoring criteria:**
    - **task_completion_score (1-10):** Did the agent successfully complete the task and produce a correct final answer?
    - **process_efficiency_score (1-10):** How direct and efficient was the process? Give HIGHER scores (7-10) when the agent: (1) laid out a full plan or list of steps *before* executing, and (2) then executed those steps in sequence with minimal back-and-forth. Give LOWER scores (1-6) when the agent: explored step-by-step with repeated think-then-search cycles, used redundant or exploratory tool calls, or could have achieved the same result with fewer, more targeted steps.
    
    **User's Task:** {query}
    **Full Agent Trace:**\n```\n{trace}\n```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Reactive Agent's Process ---")
react_agent_evaluation = evaluate_agent_process(plan_centric_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

console.print("\n--- Evaluating Planning Agent's Process ---")
planning_agent_evaluation = evaluate_agent_process(plan_centric_query, final_planning_output)
console.print(planning_agent_evaluation.model_dump())

--- Evaluating Reactive Agent's Process ---

{
    'task_completion_score': 10,
    'process_efficiency_score': 8,
    'justification': "The agent successfully completed the task by providing the population of the capital cities 
of France, Germany, and Italy, calculating their combined total, and comparing it to the population of the United 
States. The agent's process was efficient as it laid out a full plan and executed it in sequence with minimal 
back-and-forth, but could have achieved the same result with fewer, more targeted steps."
}

--- Evaluating Planning Agent's Process ---

{
    'task_completion_score': 10,
    'process_efficiency_score': 9,
    'justification': 'The agent successfully completed the task by laying out a full plan before execution and then
executing each step in sequence with minimal back-and-forth. The process was direct and efficient, with no 
redundant or exploratory tool calls.'
}

**Discussion of the Output:**
The judge is instructed to score **process_efficiency_score** by how direct and efficient the process was: higher scores (7-10) for an upfront plan followed by linear execution; lower scores (1-6) for step-by-step exploration, repeated think-tool cycles, or redundant tool calls. Both agents can receive a high `task_completion_score` if they find the answer. The **Planning Agent** should receive a significantly higher `process_efficiency_score` because its transcript shows a single planning phase then sequential execution, whereas the ReAct agent's transcript shows multiple reactive think-search-think cycles. The judge's justification should reflect this difference.

This evaluation confirms our hypothesis: for problems where the solution path is predictable, the Planning architecture offers a more structured, transparent, and efficient approach.

## Conclusion

In this notebook, we have implemented the **Planning** architecture and contrasted it directly with the **ReAct** pattern. By forcing an agent to first construct a comprehensive plan before execution, we gain significant benefits in transparency, robustness, and efficiency for well-defined, multi-step tasks.

While ReAct excels in exploratory scenarios where the next step is unknown, Planning shines when the path to a solution can be charted in advance. Understanding this trade-off is crucial for a system designer. Choosing the right architecture for the right problem is a key skill in building effective and intelligent AI agents. The Planning pattern is an essential tool in that toolkit, providing the structure needed for complex, predictable workflows.